In [20]:
from pyspark.sql import Row
from pyspark.sql.functions import col 
from pyspark.ml.linalg import Vectors

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier

#from pyspark.ml.clustering import BisectingKMeans
#from pyspark.ml.clustering import KMeans

from pyspark.mllib.clustering import KMeans, KMeansModel

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

from scipy.spatial import distance
import numpy as np
import math

import time

In [2]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

train_positive_pairs = sc.textFile(base_path + "new_positive_pairs_features/day*/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
train_unlabled_pairs = sc.textFile(base_path + "new_unlabled_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_positive_pairs = sc.textFile(base_path + "test_positive_pairs/part-*")

test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_pairs_features_50w = sc.textFile(base_path + "test_pairs_features_50w/part-*").map(lambda x: eval(x))

In [3]:
print(train_unlabled_pairs.first())

('day123-8615195968155-120.192.85.163', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 7, 2, 0, 0, 0, 0, 0, 6, 12, 72, 78, 1588, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 11, 7, 9438])


In [10]:
train_positive_pairs_np = np.array(train_positive_pairs.map(lambda x: Vectors.dense(x[1])).collect())

In [11]:
train_positive_pairs_center = np.mean(train_positive_pairs_np, axis=0)

In [12]:
print(train_positive_pairs_center)

[  6.79581979e-03   2.20344163e-02   8.72972078e-02   4.85342932e-02
   1.89400049e-02   1.00740282e-01   5.78387510e-02   5.64125203e-03
   3.82026097e-04   8.32392417e-02   3.75022285e-02   1.39121170e-01
   2.92165069e-02   4.02273480e-02   6.96391127e-02   7.13285170e-02
   3.90685355e-02   1.42453287e-01   1.65544642e-03   3.60802425e-04
   2.50439330e-03   1.52810439e-04   2.54684064e-05   3.22599815e-04
   5.09368129e-04   6.40954895e-04   2.14783561e-03   1.10363095e-04
   5.09368129e-05   8.14989006e-04   8.74415288e-04   3.82026097e-05
   1.27342032e-05   2.75003608e-01   1.73142716e-02   4.15983972e-04
   2.46194596e-04   3.32065573e-02   1.27342032e-05   1.66563378e-02
   2.54684064e-05   2.23060793e-02   9.33841570e-05   8.13376007e-02
   5.00878660e-04   1.12060988e-03   1.88169076e-02   8.91394225e-05
   1.26917559e-03   1.31756556e-02   1.09768832e-02   2.89456929e-01
   3.72475444e-02   3.54392876e-02   5.81528614e-04   3.73536628e-04
   1.44320970e-04   1.50688071e-03

In [20]:
#train_unlabled_pairs_df = spark.createDataFrame(train_unlabled_pairs.map(lambda x: Vectors.dense(x[1])), ["features"])

In [3]:
train_unlabled_pairs_features = train_unlabled_pairs.map(lambda x: x[1])

In [69]:
k = 30

In [5]:
from pyspark.mllib.clustering import KMeans, KMeansModel

kmeans_model = KMeans.train(train_unlabled_pairs_features, k)

In [6]:
centers = kmeans_model.clusterCenters

In [7]:
print(centers)

[array([  6.79380337e-03,   2.48608692e-02,   8.26590131e-02,
         4.48807836e-02,   1.63560720e-02,   9.46501211e-02,
         5.82098116e-02,   4.33408892e-03,   3.96230153e-04,
         9.29815804e-02,   3.13883750e-02,   1.43513790e-01,
         3.41542673e-02,   3.89746387e-02,   6.52763448e-02,
         6.85233737e-02,   4.16826402e-02,   1.50364197e-01,
         9.71278460e-04,   8.09184306e-04,   1.75859292e-03,
         4.02662461e-04,   4.09094768e-04,   4.01375999e-04,
         1.63251969e-03,   1.01501815e-03,   2.81606430e-03,
         2.03260923e-04,   2.04547384e-04,   1.81777015e-03,
         3.65355076e-04,   2.03260923e-04,   2.04547384e-04,
         2.97805554e-01,   2.02167430e-02,   4.07808307e-04,
         4.05235384e-04,   3.43202209e-02,   2.04547384e-04,
         1.33676218e-02,   4.09094768e-04,   2.47257907e-02,
         2.04547384e-04,   9.16565251e-02,   6.08496307e-04,
         7.87314460e-04,   1.95259132e-02,   2.04547384e-04,
         1.40095661e-03

In [8]:
len(centers)

30

In [21]:
distances = []
for vector in centers:
    dist = distance.euclidean(vector, train_positive_pairs_center)
    distances.append(dist)

In [24]:
print(len(distances), distances)

30 [89504892.64585324, 14469617.455265425, 2231120680.818343, 9903704.329421073, 392188295.8751005, 204150017.4062941, 54699457.5044344, 66473922.12206459, 47407091.09810224, 21190363.4923291, 113341098.65236531, 289835184.6863969, 42409719.014700055, 169598638.60830295, 138998819.47539365, 84734761.91902263, 86451516.34446311, 76582118.8713255, 86696846.8172044, 72904347.11240773, 48240164.04067917, 69764594.71090254, 81274318.53875503, 17256706.72011392, 88345025.42920665, 78547411.9552015, 36575592.64314324, 90330911.68035237, 26689667.659306798, 34684248.40931345]


In [ ]:
cluster_index = []
for d,j in sorted([(distances[i],i) for i in range(len(distances))], reverse=True):
    cluster_index.append(j)

In [ ]:
cluster_dict = train_unlabled_pairs_features.map(lambda x: (kmeans_model.predict(x), 1)).reduceByKey(lambda x, y : x + y).collect()

In [ ]:
m = k/2
while m > 0 and m < k:
    cluster_sum = 0
    for index in cluster_index[:m]:
        cluster_sum += cluster_dict[index][-1]
    if cluster_sum > 500000:
        m -= 1
    elif cluster_sum < 150000:
        m += 1
    else:
        break
print(m)

In [71]:
m = 7
RN = train_unlabled_pairs_features.filter(lambda x: kmeans_model.predict(x) in cluster_index[-m:])

In [72]:
RN_len = RN.count()

In [73]:
print(RN_len)

23683


In [57]:
test_pos_features = sc.textFile(base_path + "test_pos_features/part-*").map(lambda x: eval(x))
test_neg_features = sc.textFile(base_path + "test_neg_features_8w/part-*").map(lambda x: eval(x))
test_data_4thday = test_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(test_neg_features.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

In [58]:
y_label = [i.label for i in test_data_4thday.select('label').collect()]

In [65]:
test_data_4thday_length = len(y_label)
print(test_data_4thday_length)

157653


In [ ]:
Train_data = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF() 

In [ ]:
with open('qq_prediction_kmeans.log', 'a+') as log_file2:
    log_file2.write(time.asctime() + '\n')
    log_file2.write("####### k = " + str(k) + ' ########\n')
    log_file2.write("##### len of RN: " + str(RN_len) + ' ########\n\n')
    
    # LR
    log_file2.write('###### LogisticRegression #######\n')
    lr = LogisticRegression().fit(Train_data)
    predict_4th_day_LR = lr.transform(test_data_4thday.select('features'))

    y_predict_PU_LR = [i.prediction for i in predict_4th_day_LR.select('prediction').collect()]
    # print(len(y_predict_PU_LR))

    recall_score_PU_LR = recall_score(y_label, y_predict_PU_LR)
    precision_score_PU_LR = precision_score(y_label, y_predict_PU_LR)
    f1_score_PU_LR = f1_score(y_label, y_predict_PU_LR)
    tp_LR = 0
    tn_LR = 0
    for i in range(test_data_4thday_length):
        if y_label[i] == 1.0 and y_predict_PU_LR[i] == 1.0:
            tp_LR += 1
        if y_label[i] == 0.0 and y_predict_PU_LR[i] == 0.0:
            tn_LR += 1
    accuracy_LR = (tp_LR + tn_LR)/test_data_4thday_length

    print(recall_score_PU_LR, precision_score_PU_LR, f1_score_PU_LR, accuracy_LR)
    log_file2.write('\n')
    log_file2.write("recall_score_LR: " + str(recall_score_PU_LR) + '\n')
    log_file2.write("precision_score_LR: " + str(precision_score_PU_LR) + '\n')
    log_file2.write("f1_score_LR: " + str(f1_score_PU_LR) + '\n')
    log_file2.write("accuracy_LR: " + str(accuracy_LR) + '\n')
    log_file2.write(time.asctime() + '\n')
    log_file2.write('\n')

    # DT
    log_file2.write('###### DecisionTreeClassifier #######\n')
    DT = DecisionTreeClassifier().fit(Train_data)
    predict_4th_day_DT = DT.transform(test_data_4thday.select('features'))

    y_predict_PU_DT = [i.prediction for i in predict_4th_day_DT.select('prediction').collect()]

    recall_score_PU_DT = recall_score(y_label, y_predict_PU_DT)
    precision_score_PU_DT = precision_score(y_label, y_predict_PU_DT)
    f1_score_PU_DT = f1_score(y_label, y_predict_PU_DT)
    tp_DT = 0
    tn_DT = 0
    for i in range(test_data_4thday_length):
        if y_label[i] == 1.0 and y_predict_PU_DT[i] == 1.0:
            tp_DT += 1
        if y_label[i] == 0.0 and y_predict_PU_DT[i] == 0.0:
            tn_DT += 1
    accuracy_DT = (tp_DT + tn_DT)/test_data_4thday_length

    print(recall_score_PU_DT, precision_score_PU_DT, f1_score_PU_DT, accuracy_DT)

    log_file2.write('\n')
    log_file2.write("recall_score_DT: " + str(recall_score_PU_DT) + '\n')
    log_file2.write("precision_score_DT: " + str(precision_score_PU_DT) + '\n')
    log_file2.write("f1_score_DT: " + str(f1_score_PU_DT) + '\n')
    log_file2.write("accuracy_DT: " + str(accuracy_DT) + '\n')
    log_file2.write(time.asctime() + '\n')
    log_file2.write('\n')

    # RF
    log_file2.write('###### RandomForestClassifier #######\n')

    RF = RandomForestClassifier().fit(Train_data)
    predict_4th_day_RF = RF.transform(test_data_4thday.select('features'))

    y_predict_PU_RF = [i.prediction for i in predict_4th_day_RF.select('prediction').collect()]

    recall_score_PU_RF = recall_score(y_label, y_predict_PU_RF)
    precision_score_PU_RF = precision_score(y_label, y_predict_PU_RF)
    f1_score_PU_RF = f1_score(y_label, y_predict_PU_RF)
    tp_RF = 0
    tn_RF = 0
    for i in range(test_data_4thday_length):
        if y_label[i] == 1.0 and y_predict_PU_RF[i] == 1.0:
            tp_RF += 1
        if y_label[i] == 0.0 and y_predict_PU_RF[i] == 0.0:
            tn_RF += 1
    accuracy_RF = (tp_RF + tn_RF)/test_data_4thday_length

    print(recall_score_PU_RF, precision_score_PU_RF, f1_score_PU_RF, accuracy_RF)

    log_file2.write('\n')
    log_file2.write("recall_score_RF: " + str(recall_score_PU_RF) + '\n')
    log_file2.write("precision_score_RF: " + str(precision_score_PU_RF) + '\n')
    log_file2.write("f1_score_RF: " + str(f1_score_PU_RF) + '\n')
    log_file2.write("accuracy_RF: " + str(accuracy_RF) + '\n')
    log_file2.write(time.asctime() + '\n')
    log_file2.write('\n')

0.942307444658 0.772845373891 0.84920473734 0.8351633016815411


In [3]:
data = [(Vectors.dense([0.0, 0.0]),), (Vectors.dense([1.0, 1.0]),), (Vectors.dense([4.0, 5.0]),), (Vectors.dense([9.0, 8.0]),), (Vectors.dense([8.0, 9.0]),)]

In [14]:
df = spark.createDataFrame(data, ["features"])
bkm = BisectingKMeans(k=2, minDivisibleClusterSize=1.0)
model = bkm.fit(df)
centers = model.clusterCenters()

In [27]:
centers

[array([ 1.66666667,  2.        ]), array([ 8.5,  8.5])]

In [8]:
kmeans = KMeans(k=2, seed=1).fit(a)

IllegalArgumentException: 'Field "features" does not exist.'

In [18]:
kmeans

[array([ 0.5,  0.5]), array([ 7.        ,  7.33333333])]

In [23]:
KMeans(k=3, seed=1).fit(df).clusterCenters()

[array([ 8.5,  8.5]), array([ 0.5,  0.5]), array([ 4.,  5.])]

In [9]:
a = sc.parallelize(data)

In [12]:
KMeans().setK(2).setSeed(1).fit(a)

AttributeError: 'RDD' object has no attribute '_jdf'

In [10]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [34]:
data = np.array([0.0,0.0, 1.0,1.0, 9.0,8.0, 8.0,9.0]).reshape(4, 2)

In [35]:
data

array([[ 0.,  0.],
       [ 1.,  1.],
       [ 9.,  8.],
       [ 8.,  9.]])

In [36]:
model = KMeans.train(sc.parallelize(data), 2)

In [37]:
model.clusterCenters

[array([ 0.5,  0.5]), array([ 8.5,  8.5])]

In [41]:
model.predict(sc.parallelize(data).map())

1

In [43]:
sc.parallelize(data).filter(lambda x: model.predict(x)==1).collect()

[array([ 9.,  8.]), array([ 8.,  9.])]

In [13]:
from scipy.spatial import distance

In [14]:
vector1 = [1,2,3]
vector2 = [2,3,4]

In [15]:
dist = distance.euclidean(vector1, vector2)

In [16]:
dist

1.7320508075688772

In [19]:
import math
math.sqrt(3)

1.7320508075688772

In [30]:
sorted([(vector1[i],i) for i in range(len(vector1))])

[(1, 0), (2, 1), (3, 2)]